Loading neccessary library

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.feature_extraction.text import CountVectorizer

# from zipfile import ZipFile

In [0]:
# Since i have uploaded the data to my Google Drive
# I have to have that path

project_path = '/content/drive/My Drive/ML/'

In [0]:
# Since read_csv method can now read csv from zip as well
# loading the csv file

blog_text_df = pd.read_csv(project_path + 'blog-authorship-corpus.zip')

In [0]:
blog_text_df = blog_text_df.iloc[:50000]

In [0]:
# Checking the shape of of dataset

blog_text_df.shape

(50000, 7)

In [0]:
# Checking few records to look at the dataset

blog_text_df.head(8)

,id,gender,age,topic,sign,date,text
0,2059027,male,15,Student,Leo,"14,May,2004","Info has been found (+/- 100 pages,..."
1,2059027,male,15,Student,Leo,"13,May,2004",These are the team members: Drewe...
2,2059027,male,15,Student,Leo,"12,May,2004",In het kader van kernfusie op aarde...
3,2059027,male,15,Student,Leo,"12,May,2004",testing!!! testing!!!
4,3581210,male,33,InvestmentBanking,Aquarius,"11,June,2004",Thanks to Yahoo!'s Toolbar I can ...
5,3581210,male,33,InvestmentBanking,Aquarius,"10,June,2004",I had an interesting conversation...
6,3581210,male,33,InvestmentBanking,Aquarius,"10,June,2004",Somehow Coca-Cola has a way of su...
7,3581210,male,33,InvestmentBanking,Aquarius,"10,June,2004","If anything, Korea is a country o..."


In [0]:
# concise summary of dataset

blog_text_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 681284 entries, 0 to 681283
Data columns (total 7 columns):
id        681284 non-null int64
gender    681284 non-null object
age       681284 non-null int64
topic     681284 non-null object
sign      681284 non-null object
date      681284 non-null object
text      681284 non-null object
dtypes: int64(2), object(5)
memory usage: 36.4+ MB


In [0]:
# checking for the null values

blog_text_df.isna().sum()

id        0
gender    0
age       0
topic     0
sign      0
date      0
text      0
dtype: int64

In [0]:
# 
blog_text_df['sign'].value_counts()

Cancer         65048
Aries          64979
Taurus         62561
Libra          62363
Virgo          60399
Scorpio        57161
Pisces         54053
Leo            53811
Gemini         51985
Sagittarius    50036
Aquarius       49687
Capricorn      49201
Name: sign, dtype: int64

In [0]:
# Checking for the duplicate rows

duplicate_rows_df = blog_text_df[blog_text_df.duplicated(keep='first') == True]
print(duplicate_rows_df.shape)

(4686, 7)


In [0]:
duplicate_rows_df.head(20)

,id,gender,age,topic,sign,date,text
2310,589736,male,35,Technology,Aries,"05,August,2004",hey guys - i had the flu today - th...
3469,589736,male,35,Technology,Aries,"05,August,2004",
3578,589736,male,35,Technology,Aries,"05,August,2004",
3626,589736,male,35,Technology,Aries,"05,August,2004",
3627,589736,male,35,Technology,Aries,"05,August,2004",
3637,589736,male,35,Technology,Aries,"05,August,2004",
3735,3698079,female,14,indUnk,Aquarius,"21,June,2004",Around 1:00 my mom picked me ...
3736,3698079,female,14,indUnk,Aquarius,"21,June,2004",Around 1:00 my mom picked me ...
3829,3543234,male,14,indUnk,Scorpio,"07,June,2004",ZmX: I hate these geek 20 year olds...
4275,766556,female,34,indUnk,Sagittarius,"09,January,2003",Not a happy girl Working like crazy a...


In [0]:
# removing all the duplicate rows from the df

blog_text_df = blog_text_df[blog_text_df.duplicated(keep='first') == False]

In [0]:
blog_text_df

,id,gender,age,topic,sign,date,text
0,2059027,male,15,Student,Leo,"14,May,2004","Info has been found (+/- 100 pages,..."
1,2059027,male,15,Student,Leo,"13,May,2004",These are the team members: Drewe...
2,2059027,male,15,Student,Leo,"12,May,2004",In het kader van kernfusie op aarde...
3,2059027,male,15,Student,Leo,"12,May,2004",testing!!! testing!!!
4,3581210,male,33,InvestmentBanking,Aquarius,"11,June,2004",Thanks to Yahoo!'s Toolbar I can ...
...,...,...,...,...,...,...,...
49995,1273538,male,23,Advertising,Taurus,"13,August,2003",Aug 7th Thur... Bought Her Mua Chee & S...
49996,1273538,male,23,Advertising,Taurus,"13,August,2003",Aug 6th Wed.. Her 1st Day @ Work Back @...
49997,1273538,male,23,Advertising,Taurus,"13,August,2003",Aug 4th Mon Zing's BD !! Went To Her Pl...
49998,1273538,male,23,Advertising,Taurus,"13,August,2003",Aug 3rd Sun.. Went To Her Place B4 Goin...


In [0]:
#Cross check if any duplicate rows still there

blog_text_df[blog_text_df.duplicated()]

,id,gender,age,topic,sign,date,text


In [0]:
# releasing some memory
del duplicate_rows_df

In [0]:
blog_text_df['sign'].value_counts()

Aries          64758
Cancer         64709
Libra          62146
Taurus         61393
Virgo          60200
Scorpio        56597
Pisces         53657
Leo            53603
Gemini         51309
Sagittarius    49831
Aquarius       49422
Capricorn      48973
Name: sign, dtype: int64

In [0]:
blog_text_df['text'].isnull().sum()

0

In [0]:
(blog_text_df['age'].value_counts())

17    80670
24    79809
23    72448
16    72228
25    66803
26    55012
27    45745
15    41559
14    27200
34    21304
33    17532
35    17275
36    14185
13    12132
37     9280
38     7187
39     5468
40     4998
45     4468
43     4195
41     3714
48     3545
42     2875
46     2730
47     2204
44     2032
Name: age, dtype: int64

In [0]:
# blog_text_df['age_group'] = blog_text_df['age'].apply(lambda x: 'age group 1' if(x > 12 and x < 18) else ('age group 2' if(x>22 and x<28) else ('age group 3' if(x>32 and x<48) else 'age group 4')))

In [0]:
blog_text_df.head()

,id,gender,age,topic,sign,date,text
0,2059027,male,15,Student,Leo,"14,May,2004","Info has been found (+/- 100 pages,..."
1,2059027,male,15,Student,Leo,"13,May,2004",These are the team members: Drewe...
2,2059027,male,15,Student,Leo,"12,May,2004",In het kader van kernfusie op aarde...
3,2059027,male,15,Student,Leo,"12,May,2004",testing!!! testing!!!
4,3581210,male,33,InvestmentBanking,Aquarius,"11,June,2004",Thanks to Yahoo!'s Toolbar I can ...


In [0]:
import nltk;
nltk.download('stopwords');
from nltk.corpus import stopwords;
import string
stop_words = set(stopwords.words('english'))
print(stop_words)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
{'i', "it's", 'above', 'any', 'these', 'mustn', 'wasn', 've', 'yours', 'is', 'yourselves', 'before', "mightn't", 'o', 'own', 'shouldn', 'into', 'haven', 'our', 'ain', 'hasn', 'while', 'just', 'the', 'itself', 'm', 'me', 'a', 'hers', 'yourself', "you'd", 'so', 'for', 'how', 'from', "haven't", 'which', "mustn't", 'then', 'below', 'will', "hasn't", 'that', "shouldn't", "isn't", 'had', 'by', 'under', "you're", 'very', "don't", "you've", 'those', 'about', "won't", 'we', 'both', 'should', 'once', 'why', 'such', 'has', 'did', 'doing', 'wouldn', 'again', 'its', 'theirs', 'there', "you'll", 'isn', 'her', 'after', "weren't", 'than', 'what', 'you', 'been', 'most', "doesn't", 'now', 's', 'hadn', 'do', 'down', 'no', 'couldn', 'y', 'being', 'them', 'she', 'of', 'off', 'between', 'not', 'too', 'myself', "didn't", 'same', 'they', 'herself', 'when', 'out', 'ourselves', 'ours', 'where', 'are', 

In [0]:
def remove_punctuation_space_stop_words_number_lowercase(input_str):
    tokens = input_str.translate(str.maketrans("","",string.punctuation)).split()
    result = [i.lower() for i in tokens if (not i in stop_words) and (not i.isdigit())]
    return ' '.join(result)

In [0]:
blog_text_df['clean_text'] = blog_text_df['text'].apply(lambda row: remove_punctuation_space_stop_words_number_lowercase(row))

In [0]:
blog_text_df.head()

,id,gender,age,topic,sign,date,text,clean_text
0,2059027,male,15,Student,Leo,"14,May,2004","Info has been found (+/- 100 pages,...",info found pages mb pdf files now wait untill ...
1,2059027,male,15,Student,Leo,"13,May,2004",These are the team members: Drewe...,these team members drewes van der laag urllink...
2,2059027,male,15,Student,Leo,"12,May,2004",In het kader van kernfusie op aarde...,in het kader van kernfusie op aarde maak je ei...
3,2059027,male,15,Student,Leo,"12,May,2004",testing!!! testing!!!,testing testing
4,3581210,male,33,InvestmentBanking,Aquarius,"11,June,2004",Thanks to Yahoo!'s Toolbar I can ...,thanks yahoos toolbar i capture urls popupswhi...


In [0]:
blog_text_df['age'] = blog_text_df['age'].astype(str)

In [0]:
blog_text_df['labels'] = blog_text_df[['gender', 'age', 'topic', 'sign']].values.tolist()

In [0]:
blog_text_df.drop(['gender', 'age', 'topic', 'sign', 'id', 'text', 'date'], axis=1, inplace=True)
blog_text_df.head()

,clean_text,labels
0,info found pages mb pdf files now wait untill ...,"[male, 15, Student, Leo]"
1,these team members drewes van der laag urllink...,"[male, 15, Student, Leo]"
2,in het kader van kernfusie op aarde maak je ei...,"[male, 15, Student, Leo]"
3,testing testing,"[male, 15, Student, Leo]"
4,thanks yahoos toolbar i capture urls popupswhi...,"[male, 33, InvestmentBanking, Aquarius]"


In [0]:
# feature 
X = blog_text_df['clean_text']

# label
y = blog_text_df['labels']

In [0]:
del blog_text_df

In [0]:
from sklearn.model_selection import train_test_split
# import tqdm

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [0]:
print(X_train.shape)
print(y_train.shape)

(33305,)
(33305,)


In [0]:
print(X_test.shape)
print(y_test.shape)

(16404,)
(16404,)


In [0]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.33, random_state=42)

In [0]:
print(X_train.shape)
print(y_train.shape)

(22314,)
(22314,)


In [0]:
print(X_val.shape)
print(y_val.shape)

(10991,)
(10991,)


In [0]:
# del count_vect

In [0]:
# instantiate the vectorizer
count_vect = CountVectorizer(ngram_range=(1,2), min_df=2, max_features=1000, lowercase=False)
count_vect

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=False, max_df=1.0, max_features=1000, min_df=2,
                ngram_range=(1, 2), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

In [0]:
X_train_dtm = count_vect.fit_transform(X_train)

In [0]:
X_val_dtm = count_vect.transform(X_val)

In [0]:
X_test_dtm  count_vect.transform(X_test)

SyntaxError: ignored

In [0]:
lcount_vect.vocabulary_

{'dont': 200,
 'feel': 265,
 'well': 934,
 'maybe': 517,
 'got': 333,
 'everyone': 242,
 'seems': 712,
 'getting': 317,
 'sick': 737,
 'lately': 454,
 'urllink': 891,
 'the': 836,
 'of': 585,
 'by': 108,
 'wrote': 983,
 'book': 87,
 'as': 45,
 'blog': 83,
 'ill': 399,
 'note': 578,
 'from': 301,
 'one': 595,
 'american': 27,
 'history': 377,
 'however': 389,
 'states': 786,
 'state': 785,
 'five': 285,
 'next': 569,
 'president': 648,
 'united': 887,
 'america': 26,
 'it': 422,
 'written': 981,
 'sitting': 748,
 'also': 21,
 'around': 42,
 'he': 359,
 'there': 838,
 'process': 654,
 'part': 611,
 'perhaps': 620,
 'means': 520,
 'but': 106,
 'interest': 413,
 'happened': 352,
 'interesting': 415,
 'going': 329,
 'later': 455,
 'brother': 98,
 'birthday': 79,
 'basically': 63,
 'enjoy': 229,
 'cool': 159,
 'air': 13,
 'thats': 835,
 'plan': 632,
 'not': 577,
 'park': 610,
 'scared': 704,
 'nice': 570,
 'shopping': 730,
 'like': 473,
 'first': 283,
 'world': 970,
 'couldnt': 161,
 'get': 

In [0]:
from sklearn.preprocessing import MultiLabelBinarizer

In [0]:
multi_label_binarizer = MultiLabelBinarizer()

In [0]:
y_train_t =  multi_label_binarizer.fit_transform(y_train)

In [0]:
multi_label_binarizer.classes_

array(['13', '14', '15', '16', '17', '23', '24', '25', '26', '27', '33',
       '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44',
       '45', '46', '47', '48', 'Accounting', 'Advertising', 'Agriculture',
       'Aquarius', 'Architecture', 'Aries', 'Arts', 'Automotive',
       'Banking', 'Biotech', 'BusinessServices', 'Cancer', 'Capricorn',
       'Chemicals', 'Communications-Media', 'Construction', 'Consulting',
       'Education', 'Engineering', 'Environment', 'Fashion', 'Gemini',
       'Government', 'HumanResources', 'Internet', 'InvestmentBanking',
       'Law', 'LawEnforcement-Security', 'Leo', 'Libra', 'Manufacturing',
       'Maritime', 'Marketing', 'Military', 'Museums-Libraries',
       'Non-Profit', 'Pisces', 'Publishing', 'RealEstate', 'Religion',
       'Sagittarius', 'Science', 'Scorpio', 'Sports-Recreation',
       'Student', 'Taurus', 'Technology', 'Telecommunications', 'Tourism',
       'Transportation', 'Virgo', 'female', 'indUnk', 'male'],
      dtype

In [0]:
y_val_t = multi_label_binarizer.transform(y_val)

In [0]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression

In [0]:
clf = OneVsRestClassifier(LogisticRegression(solver = 'sag', max_iter=7000))

In [0]:
clf.fit(X_train_dtm, y_train_t)

OneVsRestClassifier(estimator=LogisticRegression(C=1.0, class_weight=None,
                                                 dual=False, fit_intercept=True,
                                                 intercept_scaling=1,
                                                 l1_ratio=None, max_iter=7000,
                                                 multi_class='auto',
                                                 n_jobs=None, penalty='l2',
                                                 random_state=None,
                                                 solver='sag', tol=0.0001,
                                                 verbose=0, warm_start=False),
                    n_jobs=None)

In [0]:
clf.score(X_val_dtm, y_val_t)

0.015103266308798107

In [0]:
import pickle
filename = 'finalized_model.sav'
pickle.dump(clf, open(filename, 'wb'))

In [0]:
loaded_model = pickle.load(open(filename, 'rb'))
loaded_model.score(X_val_dtm, y_val_t)
# print(result)

0.015103266308798107

In [0]:
blog_text_df['clean_text'] = blog_text_df['clean_text'].apply(lambda x: )

In [0]:
import string
input1 = blog_text_df['text'][3]
' '.join(row.translate(str.maketrans("","",string.punctuation)).split()).lower()

'testing testing'

In [0]:
aa = blog_text_df['text'][:6]
aa = aa.apply(lambda row: remove_punctuation_space_stop_words_lowercase(row))
aa[3]

'testing testing'

In [0]:
from nltk.tokenize import word_tokenize
def remove_punctuation_space_stop_words_lower(input_str):
    tokens = input_str.translate(str.maketrans("","",string.punctuation)).split() #word_tokenize(input_str)
    result = [i for i in tokens if not i in stop_words]
    return ' '.join(result)

In [0]:
remove_punctuation_space_stop_words_number_lowercase(aa1)

'info found pages mb pdf files now wait untill team leader processed learns html'

In [0]:
aa[600000]

'prev urllink vol i urllink vol ii urllink vol iii this is based on a true story names places and dates have been changed to protect the innocent and in the case of one guy the very stupid some scenes have been dramatized for that extra oomph matt had to work for everything he got life never gave him an even break his mother died when he was still an infant and his father a man who couldnt tell the difference between a diaper and a rag left him at his maternal grandparents home his grandfather worked as a newspaper deliveryman while his grandmother scrubbed toilets for wealthy homeowners matt could see the hard work they put in day in and day out only to reap measly financial returns he knew at a very young age that he would never become like them he would never be poor and he would do anything to avoid such a life matt was a sensible young lad he never dreamed of striking it rich via lottery tickets or by getting a stable job in the government sector and work until he retires so he co

In [0]:
import re
aa1 = blog_text_df['text'][0]
re.split(r'\d+', aa1)
# ' '.join(aa1.split())

['           Info has been found (+/- ',
 ' pages, and ',
 '.',
 ' MB of .pdf files) Now i have to wait untill our team leader has processed it and learns html.         ']

In [0]:
# instantiate the vectorizer
vect = CountVectorizer()

In [0]:
vect.fit(blog_text_df['text'])
X_train_dtm = vect.transform(blog_text_df['text'])

In [0]:
X_train_dtm

NameError: ignored